# Consulta de documentos con RAG con Ollama, Langchain y llama2



<p align="center">
<img src ="logoSena.png">
</p>

## LLM's corriendo a nivel local.

Aprendizaje - Barato - Bajo poder para tareas especificas - Privacidad
Robotica - Edge AI 

# Ollama
Permite correr LLM's de manera local.  
+ Vamos a bajar el modelo y dejarlo en local, ahi tenemos una interfaz completa para interactuar con el modelo.

+ Available Commands:
  + serve       Start ollama
  + create      Create a model from a Modelfile
  + show        Show information for a model
  + run         Run a model
  + pull        Pull a model from a registry
  + push        Push a model to a registry
  + list        List models
  + cp          Copy a model
  + rm          Remove a model
  + help        Help about any command



In [2]:
# Librerias a usar

# Entorno virtual (conda, miniconda, poetry), yo uso venv
# Key's para conexión a servicios en .env OPENAI_API_KEY= sk.....
# python_dotenv
# langchain_openai (Me permite conectarme con la api de openai)
# 

In [37]:
import os
from dotenv import load_dotenv

load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
MODEL = "llama2"

In [4]:
# Libreria
# langchain-community

In [38]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key=OPENAI_API_KEY, model = MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model = MODEL)    
    embeddings = OllamaEmbeddings()


model.invoke("Cuentame un chiste")

'¡Claro! Aquí tienes uno:\n\n¿Por qué el león siempre gana en el ajedrez?\nPorque siempre come al alfoz!'

### Langchain soporta el concepto de parser (segmentador?)
En caso que usemos chatgpt, ya que envia el mensaje de salida como una salida de chat con una metadata de chat  
AIMesagge....

En cambio llama2 es un completion model y devuelve un string.

In [9]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

#Primer langchain, tomar el modelo y unir la salida del modelo con la entrada del parseador
chain = model | parser 
# Ya no invoco (llamo) al modelo, ahora llamo a la cadena
chain.invoke("Cuentame un chiste")

'Sí, ¡claro! Aquí tienes uno:\n\n¿Por qué el león siempre gana en las carreras?\nPorque siempre come a los demás concursantes antes de la carrera... ¡ya veis! 😂'

### Langchain soporta loaders de documentos, y permite cargarlo y separarlo (load and split)

In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("grafo.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='HomeMy Network Jobs MessagingNotiﬁcations Me\nFor BusinessNetwork Smarter,Try Premium Free11Create your own newsletter\nStart your own discussion with a newsletter on LinkedIn. Share what you know and build\nyour thought leadership with every new edition.\nTry it out(2) What is Graph Theory, and why should you care? | ... https://www.linkedin.com/pulse/what-graph-theory-wh...\n1 of 10 5/27/24, 22:07', metadata={'source': 'grafo.pdf', 'page': 0}),
 Document(page_content='What is Graph Theory,\nand why should you\ncare?A graph visualization issued from the Opte Project, a tentative cartography of the Internet:\nhttps://en.wikipedia.org/wiki/Opte_Project\nVegard Flovik, PhD9 articlesFollow\nAugust 13, 2020Open Immersive Reader\nFrom graph theory to path\noptimization\nGraph theory  might sound like an intimidating and abstract topic to you,\nso why should you even spend your time reading an article about it?\nHowever, although it might not sound very applicable, th

### El siguiente paso es crear un template (promp template)

In [22]:
from langchain.prompts import PromptTemplate

# Langchain me permite mandar variables en las consultas, asi que creo una consulta donde voy a enviar mis documentos como contexto

template = """"

Answer the question based on the context below.  If you can't
answer the question, reply "I don't know'


Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))




"

Answer the question based on the context below.  If you can't
answer the question, reply "I don't know'


Context: Here is some context

Question: Here is a question



### ¿Cómo le paso este prompt al modelo? con una cadena

In [23]:
chain = prompt | model | parser 
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

### Ahora invoco la cadena que monté pasandole el contexto y la pregunta

In [25]:
chain.invoke(
    {
        "context":"El nombre que me fue dado fue Gabriel",
        "question": "Cual es mi nombre?"
    }
)

' Ah, an easy one! Based on the context you provided, your name is Gabriel.'

### Necesitamos pasar SOLAMENTE lo relevante como contexto (Por aquello del RAG)
Usamos una base de datos vectorial muy sencilla que va a almacenar la información de pages no como un documento sino que la va a volver embeddings, para comparar esos embeddings con la pregunta que el usuario hace y buscar los similares a la pregunta para determinar donde esta la información acertada.

In [28]:
# Instalamos
# docarray
#Una versión espacífica de pydantic==1.10.8
# 'langchain[docarray]'

### Cada MODEL me va a exigir una forma diferente de generar embeddings, tanto chat gpg como llama, por eso usaba
---
if MODEL.startswith("gpt"):

    model = ChatOpenAI(api_key=OPENAI_API_KEY, model = MODEL)
    
    embeddings = OpenAIEmbeddings()
else:

    model = Ollama(model = MODEL)    
    
    embeddings = OllamaEmbeddings()

In [30]:
from langchain_community.vectorstores import DocArrayInMemorySearch

#Creo un vectorstore en memoria, solo para motivos del ejercicio, para algo permanente
# Mejor usar almacenamiento permanente, algo como pinecone

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)
# vectorstore

### Ahora llamamos el vectorstore y le mandamos una consulta

In [35]:
# El retriver es un elemento de langchain que nos permite recuperar información de cualquier parte
retriever = vectorstore.as_retriever()
retriever.invoke("Bridge")

#Me va a entregar los documentos más relevantes tema enviado

[Document(page_content='some of you in solving similar problems later, or at least satisfy some ofyour curiosity when it comes to graph theory and some of its\napplications.\nThe cases discussed in the article covers just a few examples that\nillustrate some of the possibilities that exist. If you have previous\nexperience and ideas on the topic, it would be interesting to hear your\nthoughts in the comments below!\nIf you found the article interesting, you might also enjoy some of my other\nposts on various topics related to AI, Machine Learning, Data Science, etc.,\navailable on my medium author proﬁle\n(This post was originally published on towardsdatascience.com  )\nReport this\nPublished by\nVegard Flovik, PhDVP AI & Data Science at Aize | Associate Professor in Data SciencePublished • 3y9\narticlesFollow\nBack from vacation --> time for a new blog post!\nGraph theory might sound like an intimidating and abstract topic to you, so why\nshould you even spend your time reading an art

In [46]:
from operator import itemgetter
#itemgetter me permite llamar el contenido de un diccionario

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }    
    | prompt
    | model
    | parser
)

chain.invoke({"question":"Weighted graphs are directed graphs?"})

'No, weighted graphs are not necessarily directed graphs. A weighted graph is a graph that has additional information associated with each edge, known as the weight or cost of traveling between the two nodes connected by that edge. The weights can be any type of number, such as distance, time, or money.\n\nA directed graph, on the other hand, is a graph in which the edges have a direction, indicating the direction of travel along each edge. In a directed graph, edges are often represented as arrows, with the tail of the arrow representing the starting node and the head representing the ending node.\n\nWhile weighted graphs can be directed graphs, not all directed graphs are weighted graphs. For example, a graph consisting of only undirected edges without any weights would still be considered an undirected graph. Similarly, a graph with both directed and undirected edges could be considered a mixed graph or a hybrid graph, but it is not necessarily a weighted graph.\n\nIn the article yo

In [55]:
questions = [
    "What types of graphs the article mention",
    "What's the traveling salesman problem?",
    "What's a method that can solve the warehouse problem?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What types of graphs the article mention
Answer: The article mentions several types of graphs, including:

1. Undirected Graphs: In this type of graph, there is no direction associated with the edges, and nodes can be reached from any other node without any orientation.
2. Directed Graphs (DiGraphs): Unlike undirected graphs, directed graphs have orientation or direction among different nodes. Edges show the bidirectional roads.
3. Weighted Graphs: Many graphs can have edges containing a weight associated to represent a real-world implication such as cost, distance, quantity etc… These weighted graphs are commonly used to program GPS’s and travel-planning search engines that compare flight times and costs.
4. Routing Graphs: This problem can be formulated as an optimization problem in graph theory. All pickup points in the warehouse form a “node” in the graph, where the edges represent permitted lanes/corridors and distances between the nodes. Simple enough, right? Being able

## ¿Cómo hacemos una respuesta "bonita", que parezca generada por un LLM comercial?
### Usamos un stream

In [57]:
for s in chain.stream({"question": "What is the purpose of the article?"}):
    print(s, end="", flush=True)

The purpose of this article appears to be twofold:

1. To introduce the concept of graph theory and explain why it is relevant and useful in solving real-world problems, particularly in the context of a warehouse picking list.
2. To provide a simple example of how graph theory can be applied to solve a logistics problem in a warehouse setting. The author explains how the problem can be formulated as an optimization problem in graph theory and how mathematical techniques known from graph theory can be used to find the optimal "driving route" between the various nodes in the warehouse.

Overall, the article seems to aim at demonstrating the potential of graph theory in addressing logistics problems and encouraging readers to explore the topic further.

## Ahora una respuesta en batch para alimentar algun otro producto
### Batch


In [58]:
chain.batch([{"question": q} for q in questions])

['The article mentions several types of graphs, including:\n\n1. Undirected Graphs: A graph where there is no direction associated with edges. Each node can be reached from any other node without any orientation or direction.\n2. Directed Graphs (DiGraphs): A graph where the orientation or direction of edges is specified. There is a beginning and end to each edge, and you can move only in one direction.\n3. Weighted Graphs: A graph where each edge has a weight or cost associated with it. These graphs are commonly used to program GPS\'s and travel-planning search engines that compare flight times and costs.\n4. Shortest Route Graph Theory: This problem can be formulated as an optimization problem in graph theory, where all pickup points in the warehouse form a "node" in the graph, with edges representing permitted lanes/corridors and distances between the nodes.',
 'The traveling salesman problem is a classic problem in combinatorial optimization and operations research that involves fi